In [49]:
log1 = r'C:\Users\User\Desktop\Final_ASV_Results\logits_ASV_Default_32_Or.txt'
log2 = r'C:\Users\User\Desktop\Final_ASV_Results\logits_ASV_Special_32_Or.txt'


log_aug_def = r'C:\Users\User\Desktop\Final_ASV_Results\logits_ASV_Default_32_Aug_5.txt'
log_aug_special = r'C:\Users\User\Desktop\Final_ASV_Results\logits_ASV_Special_32_Aug_5.txt'

import numpy as np

# Load the logits from the text files
logits1 = np.loadtxt(log1)  # Replace with your file path
logits2 = np.loadtxt(log2)  # Replace with your file path

# Ensure both arrays have the same shape
if logits1.shape != logits2.shape:
    raise ValueError("The shapes of the two logits arrays must be the same.")

# Calculate the mean per row
mean_logits = np.mean(np.array([logits1, logits2]), axis=0)

# Convert mean logits to a list
mean_logits_list = mean_logits.tolist()

# Print the mean logits list
print("Mean Logits per Row as a List:")
print(mean_logits_list[0:10])



''' 
# Save the mean logits list to a text file
with open('mean_logits.txt', 'w') as f:
    for logit in mean_logits_list:
        f.write(f"{logit}")
'''

Mean Logits per Row as a List:
[-15.661702632904053, -8.33368444442749, -11.432767033576965, -10.726195096969604, -16.38002872467041, -26.487271308898926, -9.288040161132812, 16.882977962493896, -2.255512073636055, -10.694772243499756]


' \n# Save the mean logits list to a text file\nwith open(\'mean_logits.txt\', \'w\') as f:\n    for logit in mean_logits_list:\n        f.write(f"{logit}")\n'

In [50]:
import torch
import torch.nn.functional as F



def compute_predictions(logits, threshold=0.5):
    # Apply sigmoid to convert logits to probabilities (binary classification)
    probabilities = torch.sigmoid(logits)
    predictions = (probabilities > threshold).float()
    return predictions



preds = compute_predictions(torch.tensor(mean_logits_list))
preds_list = [int(x) for x in preds.tolist()]

print(preds_list[0:10])

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]


### This time you do not have the metadata linked to a File_ID name and thus you must be sure that each row contains the file you want. We know that the data/flac file are parsed in a "sorted" matter and that is why we choose to use the SORTD version of the metadata. 

The order can be locate from the protocol.txt file

In [51]:
# Ground Truth from the meta-data of eval


ground_truth_path = r'C:\Users\User\Desktop\Processed_ASV_Data\Sorted_Metadata\eval_meta_sort.txt'
#ground_truth_path = r'C:\Users\User\Desktop\Processed_ASV_Data\Eval_CM_2021_Metadata\correct_eval_metadata.txt'


# Initialize an empty list to store the last elements (spoof or bonafide)
ground_truth_labels = []

# Open the file and read each line
with open(ground_truth_path, 'r') as file:  # Replace 'protocol.txt' with your actual file path
    for line in file:
        # Split the line by spaces (or tabs if your file uses tabs)
        elements = line.split()
        
        # Append the last element (spoof or bonafide) to the list
        ground_truth_labels.append(elements[-1])

print(ground_truth_labels[0:10])

# Convert 'spoof' to 0 and 'bonafide' to 1
ground_truth_numeric = [0 if label == 'spoof' else 1 for label in ground_truth_labels]

print(ground_truth_numeric[0:10])

# preds_label = [1,0,1,1,...]
# ground_truth = [spoof, bonafide, bonafide,...]
# ground_label = [0,1,1,1,....]

['spoof', 'spoof', 'spoof', 'spoof', 'spoof', 'spoof', 'spoof', 'bonafide', 'spoof', 'spoof']
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]


In [52]:
# Find the False Positive and False Negative between [prediction] and [true_labels] 
# to find the EER

import os
import glob
import pandas as pd
from sklearn.metrics import roc_curve, roc_auc_score, det_curve
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

###############################
# Step 3: Calculating Metrics #
###############################
def calculate_metrics(y_true, y_scores):
    # y_true: Ground truth labels (0 or 1)
    # y_scores: Predicted scores or probabilities

    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_threshold = thresholds[np.nanargmin(np.absolute(fnr - fpr))]
    eer = fpr[np.nanargmin(np.absolute(fnr - fpr))]

    auc_roc = roc_auc_score(y_true, y_scores)

    det_fpr, det_fnr, _ = det_curve(y_true, y_scores)

    return eer, auc_roc, det_fpr, det_fnr

'''
# Example usage with predicted binary labels and ground truth:
ground_truth_numeric = [1, 0, 1, 1, 0, 1, 0, 0, 1, 0]  # Replace with your actual ground truth list
pred_list = [0.9, 0.1, 0.85, 0.95, 0.4, 0.7, 0.2, 0.1, 0.99, 0.5]  # Replace with your predicted scores/probabilities
'''


# Ensure both lists are NumPy arrays
y_true = np.array(ground_truth_numeric)
y_scores = np.array(preds_list)

# Calculate metrics
eer, auc_roc, det_fpr, det_fnr = calculate_metrics(y_true, y_scores)

# Print results
print(f"EER: {eer}")
print(f"AUC-ROC: {auc_roc}")

EER: 0.1243792684870704
AUC-ROC: 0.8730748357326192


In [54]:
log_origin_def_path = r'C:\Users\User\Desktop\Final_ASV_Results\logits_ASV_Default_32_Or.txt'
logits1 = np.loadtxt(log_origin_def_path)

log_origin_special_path = r'C:\Users\User\Desktop\Final_ASV_Results\logits_ASV_Special_32_Or.txt'
logits2 = np.loadtxt(log_origin_special_path)

log_aug_def_path = r'C:\Users\User\Desktop\Final_ASV_Results\logits_ASV_Default_32_Aug_5.txt'
logits3 = np.loadtxt(log_aug_def_path)

log_aug_special_path = r'C:\Users\User\Desktop\Final_ASV_Results\logits_ASV_Special_32_Aug_5.txt'
logits4 = np.loadtxt(log_aug_special_path)

# Run each model by its own:

## Original Dataset

### Default Model

In [55]:
logits_list = logits1.tolist()

preds = compute_predictions(torch.tensor(logits_list))
preds_list = [int(x) for x in preds.tolist()]

# Convert 'spoof' to 0 and 'bonafide' to 1
ground_truth_numeric = [0 if label == 'spoof' else 1 for label in ground_truth_labels]


# Ensure both lists are NumPy arrays
y_true = np.array(ground_truth_numeric)
y_scores = np.array(preds_list)

# Calculate metrics
eer, auc_roc, det_fpr, det_fnr = calculate_metrics(y_true, y_scores)
# Print results
print(f"EER: {eer}")
print(f"AUC-ROC: {auc_roc}")

EER: 0.11284745638020036
AUC-ROC: 0.8802498031344176


### Special Model

In [56]:
logits_list = logits2.tolist()

preds = compute_predictions(torch.tensor(logits_list))
preds_list = [int(x) for x in preds.tolist()]

# Convert 'spoof' to 0 and 'bonafide' to 1
ground_truth_numeric = [0 if label == 'spoof' else 1 for label in ground_truth_labels]


# Ensure both lists are NumPy arrays
y_true = np.array(ground_truth_numeric)
y_scores = np.array(preds_list)

# Calculate metrics
eer, auc_roc, det_fpr, det_fnr = calculate_metrics(y_true, y_scores)
# Print results
print(f"EER: {eer}")
print(f"AUC-ROC: {auc_roc}")

EER: 0.13502213176060915
AUC-ROC: 0.8568331786460339


## Augmented Dataset

### Default Model

In [57]:
logits_list = logits3.tolist()

preds = compute_predictions(torch.tensor(logits_list))
preds_list = [int(x) for x in preds.tolist()]

# Convert 'spoof' to 0 and 'bonafide' to 1
ground_truth_numeric = [0 if label == 'spoof' else 1 for label in ground_truth_labels]


# Ensure both lists are NumPy arrays
y_true = np.array(ground_truth_numeric)
y_scores = np.array(preds_list)

# Calculate metrics
eer, auc_roc, det_fpr, det_fnr = calculate_metrics(y_true, y_scores)
# Print results
print(f"EER: {eer}")
print(f"AUC-ROC: {auc_roc}")

EER: 0.1496315460352882
AUC-ROC: 0.8650823410079358


### Special Model

In [58]:
logits_list = logits4.tolist()

preds = compute_predictions(torch.tensor(logits_list))
preds_list = [int(x) for x in preds.tolist()]

# Convert 'spoof' to 0 and 'bonafide' to 1
ground_truth_numeric = [0 if label == 'spoof' else 1 for label in ground_truth_labels]


# Ensure both lists are NumPy arrays
y_true = np.array(ground_truth_numeric)
y_scores = np.array(preds_list)

# Calculate metrics
eer, auc_roc, det_fpr, det_fnr = calculate_metrics(y_true, y_scores)
# Print results
print(f"EER: {eer}")
print(f"AUC-ROC: {auc_roc}")

EER: 0.12098287087558394
AUC-ROC: 0.8082761778290626


# Combine logits per dataset (Default + Special)

### Original Dataset

In [60]:
# Ensure both arrays have the same shape
if logits1.shape != logits2.shape:
    raise ValueError("The shapes of the two logits arrays must be the same.")

#  Average of Logits (which gives equal weight to each logit)
mean_logits_origin = np.mean(np.array([logits1, logits2]), axis=0)


#  Weighted Average of Logits (assign different weights to each set of logits based on their performance)
weights = np.array([0.8, 0.2])  # Example weights, tune based on model performance
mean_logits_origin = np.average(np.array([logits1, logits2]), axis=0, weights=weights)


# Max Pooling (Element-wise Max)
mean_logits_origin = np.maximum(logits1, logits2)

# Product of Logits
mean_logits_origin = np.prod(np.array([logits3, logits4]), axis=0)

# Harmonic Mean (For situations where low values in any of the logits should have a bigger effect)
mean_logits_origin = 2 / (1 / logits1 + 1 / logits2)


# Learned Weighting (Ensemble Learning)
# combined_logits = model([logits3, logits4])  # Train a separate model to combine the logits






# Convert mean logits to a list
mean_logits_origin_list = mean_logits_origin.tolist()

preds = compute_predictions(torch.tensor(mean_logits_origin_list))
preds_list = [int(x) for x in preds.tolist()]

# Convert 'spoof' to 0 and 'bonafide' to 1
ground_truth_numeric = [0 if label == 'spoof' else 1 for label in ground_truth_labels]


# Ensure both lists are NumPy arrays
y_true = np.array(ground_truth_numeric)
y_scores = np.array(preds_list)

# Calculate metrics
eer, auc_roc, det_fpr, det_fnr = calculate_metrics(y_true, y_scores)
# Print results
print(f"EER: {eer}")
print(f"AUC-ROC: {auc_roc}")

EER: 0.1234903196537391
AUC-ROC: 0.8640081460478324


### Augmented Dataset

In [61]:
# Ensure both arrays have the same shape
if logits3.shape != logits4.shape:
    raise ValueError("The shapes of the two logits arrays must be the same.")

# Calculate the mean per row
mean_logits_augmented = np.mean(np.array([logits3, logits4]), axis=0)

# Convert mean logits to a list
mean_logits_augmented_list = mean_logits_augmented.tolist()

preds = compute_predictions(torch.tensor(mean_logits_augmented_list))
preds_list = [int(x) for x in preds.tolist()]

# Convert 'spoof' to 0 and 'bonafide' to 1
ground_truth_numeric = [0 if label == 'spoof' else 1 for label in ground_truth_labels]


# Ensure both lists are NumPy arrays
y_true = np.array(ground_truth_numeric)
y_scores = np.array(preds_list)

# Calculate metrics
eer, auc_roc, det_fpr, det_fnr = calculate_metrics(y_true, y_scores)
# Print results
print(f"EER: {eer}")
print(f"AUC-ROC: {auc_roc}")

EER: 0.13087779099280258
AUC-ROC: 0.8290440873780838


## Learned Weighting (Ensemble Learning)

In [103]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


log_origin_def_path = r'C:\Users\User\Desktop\Final_ASV_Results\logits_ASV_Default_32_Or.txt'
logits1 = np.loadtxt(log_origin_def_path)

log_origin_special_path = r'C:\Users\User\Desktop\Final_ASV_Results\logits_ASV_Special_32_Or.txt'
logits2 = np.loadtxt(log_origin_special_path)

# Assuming logits1 and logits2 are already loaded as numpy arrays and reshaped appropriately
# Convert logits1 and logits2 to float32 tensors
logits1 = torch.tensor(logits1).reshape(-1, 1).float()  # Ensure it's float32
logits2 = torch.tensor(logits2).reshape(-1, 1).float()  # Ensure it's float32

# Concatenate logits1 and logits2 to form the input features
inputs = torch.cat((logits1, logits2), dim=1)

# Assuming y_true is your true labels
y_true = torch.randint(0, 2, (inputs.size(0),)).float()  # Ensure y_true is float32

# Define a simple neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 64)      # Increase hidden units
        self.fc2 = nn.Linear(64, 32)     # Additional layer
        self.fc3 = nn.Linear(32, 16)     # Additional layer
        self.fc4 = nn.Linear(16, 1)
        self.dropout = nn.Dropout(0.3)   # Dropout layer to prevent overfitting
        self.batch_norm = nn.BatchNorm1d(64)  # Batch normalization layer

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.batch_norm(x)          # Apply batch normalization
        x = F.relu(self.fc2(x))
        x = self.dropout(x)             # Apply dropout
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))  # Sigmoid for binary classification
        return x

# Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.003)

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()

    # Forward pass
    outputs = model(inputs).squeeze()  # Get the outputs (predictions)
    loss = criterion(outputs, y_true)  # Calculate the loss

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update the model parameters

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# After training, you can evaluate the model on new data or test logits

Epoch [10/100], Loss: 0.6943
Epoch [20/100], Loss: 0.6938
Epoch [30/100], Loss: 0.6934
Epoch [40/100], Loss: 0.6934
Epoch [50/100], Loss: 0.6933
Epoch [60/100], Loss: 0.6933
Epoch [70/100], Loss: 0.6933
Epoch [80/100], Loss: 0.6932
Epoch [90/100], Loss: 0.6932
Epoch [100/100], Loss: 0.6932


In [104]:

'''
# To be fair, we must select a SEPERATE dataset to test the model (A subset of "logits" and "y_true")
logits1_test = torch.tensor(logits1_test).reshape(-1, 1).float()  # Replace with your test data
logits2_test = torch.tensor(logits2_test).reshape(-1, 1).float()
'''

# Set the model to evaluation mode
model.eval()

# Example test data (logits1_test, logits2_test)
# Ensure your test data is processed the same way as the training data
logits1_test = torch.tensor(logits1).reshape(-1, 1).float()  # Replace with your test data
logits2_test = torch.tensor(logits2).reshape(-1, 1).float()

# Concatenate test logits
test_inputs = torch.cat((logits1_test, logits2_test), dim=1)

# Get model predictions
with torch.no_grad():  # No need to calculate gradients during evaluation
    test_outputs = model(test_inputs).squeeze()

# Convert probabilities to binary predictions (0 or 1)
predictions = (test_outputs >= 0.5).float()

predictions_list = [int(p) for p in predictions]
print("Predictions:", predictions_list)

C:\Users\User\AppData\Local\Temp\ipykernel_12508\678954702.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  logits1_test = torch.tensor(logits1).reshape(-1, 1).float()  # Replace with your test data
C:\Users\User\AppData\Local\Temp\ipykernel_12508\678954702.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  logits2_test = torch.tensor(logits2).reshape(-1, 1).float()


Predictions: [1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,

In [105]:
# Ensure both lists are NumPy arrays
y_true = np.array(ground_truth_numeric)
y_scores = predictions_list

# Calculate metrics
eer, auc_roc, det_fpr, det_fnr = calculate_metrics(y_true, y_scores)
# Print results
print(f"EER: {eer}")
print(f"AUC-ROC: {auc_roc}")

EER: 0.7757887121890211
AUC-ROC: 0.5970937210786956
